In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer

In [2]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to C:\Users\Arjun
[nltk_data]     V\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df = pd.read_csv("train.csv")

In [4]:
df.head()

,id,comment_text,toxic
0,e617e2489abe9bca,"""\r\n\r\n A barnstar for you! \r\n\r\n The De...",0
1,9250cf637294e09d,"""\r\n\r\nThis seems unbalanced. whatever I ha...",0
2,ce1aa4592d5240ca,"Marya Dzmitruk was born in Minsk, Belarus in M...",0
3,48105766ff7f075b,"""\r\n\r\nTalkback\r\n\r\n Dear Celestia... """,0
4,0543d4f82e5470b6,New Categories \r\n\r\nI honestly think that w...,0


In [5]:
df["tokenized"] = df["comment_text"]
df = df.apply(lambda x: x.str.lower() if x.dtype == "object" else x) 
df["tokenized"] = df['tokenized'].str.replace('[^\w\s]','')
df["tokenized"] = df["tokenized"].apply(word_tokenize)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df["tokenized"] = df["tokenized"].apply(lambda words: [word for word in words if word not in stop_words])
stemmer = PorterStemmer()
df['stemmed'] = df['tokenized'].apply(lambda x: [stemmer.stem(y) for y in x])

C:\Users\ARJUNV~1\AppData\Local\Temp/ipykernel_39092/1912185540.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["tokenized"] = df['tokenized'].str.replace('[^\w\s]','')
[nltk_data] Downloading package stopwords to C:\Users\Arjun
[nltk_data]     V\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df.head()

,id,comment_text,toxic,tokenized,stemmed
0,e617e2489abe9bca,"""\r\n\r\n a barnstar for you! \r\n\r\n the de...",0,"[barnstar, defender, wiki, barnstar, like, edi...","[barnstar, defend, wiki, barnstar, like, edit,..."
1,9250cf637294e09d,"""\r\n\r\nthis seems unbalanced. whatever i ha...",0,"[seems, unbalanced, whatever, said, mathsci, s...","[seem, unbalanc, whatev, said, mathsci, said, ..."
2,ce1aa4592d5240ca,"marya dzmitruk was born in minsk, belarus in m...",0,"[marya, dzmitruk, born, minsk, belarus, march,...","[marya, dzmitruk, born, minsk, belaru, march, ..."
3,48105766ff7f075b,"""\r\n\r\ntalkback\r\n\r\n dear celestia... """,0,"[talkback, dear, celestia]","[talkback, dear, celestia]"
4,0543d4f82e5470b6,new categories \r\n\r\ni honestly think that w...,0,"[new, categories, honestly, think, need, add, ...","[new, categori, honestli, think, need, add, ca..."


In [7]:
my_dict = pd.Series(df.stemmed.values,index=df.id).to_dict()
my_list = df["stemmed"]
my_list[:5]
words = []
for key in my_dict:
    for i in range(len(my_dict[key])):
        term = my_dict[key][i]
        words.append(term)

In [8]:
print(words[:5])

['barnstar', 'defend', 'wiki', 'barnstar', 'like']


In [9]:
length = len(my_list)
inverted_index = {}
for i in range(length):
    check = my_list[i]
    for item in words:
        if item in check:
            if item not in inverted_index:
                inverted_index[item] = []
            if item in inverted_index:
                inverted_index[item].append(i+1)
        
for key in inverted_index:
    inverted_index[key] = list(set(inverted_index[key]))

In [10]:
inverted_index["minsk"]

[3]

In [16]:
complete_words=list(words)

In [17]:
def spell_correction(tokens,word):
  edit_dist=10000000
  for token in tokens:
    dist=nltk.edit_distance(token,word)
    if dist<edit_dist:
      ans=token
      edit_dist=dist
    elif dist==edit_dist:
      if len(inverted_index[token])>len(inverted_index[ans]):
        ans=token
  return ans

In [19]:
user_query=input("Enter the word : ")
result=spell_correction(complete_words,user_query)
print("The closest word is:-")
print(result)


Enter the word : chickun
The closest word is:-
chicken
